In [1]:
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt
import csv
import pandas as pd

In [4]:
#initialize parameters
gamma=10**(-8)
lbd=10**8
M=10**4
h=10**(-4)
m0=0
k=10**18

In [3]:
#function to perform stochastic gradient langevin descent
def sgld(X,M,h,gamma,lbd,n_sim,c):
    ms=np.zeros(M)
    ms[0]=m0
    dB = np.random.randn(M)   
    for i in range(1,M):
        dL=1-1/c*(np.sum(X>=ms[i-1]))/n_sim
        ms[i]= ms[i-1]-(dL+gamma*ms
                        [i-1])*h + np.sqrt(2*lbd**(-1)*h)*dB[i]
    return ms

#function to compute the average value at risk
def avar(X,M,h,gamma,lbd,n_sim,c):
    var=sgld(X,M,h,gamma,lbd,n_sim,c)
    values=np.zeros((n_sim,M))
    for i in range(n_sim):
        values[i]=np.amax(((X[i]-var),np.zeros(M)),axis=0)
    return np.mean(values/c,axis=0)+var+gamma/2*var**2

In [ ]:
n_sim=5000
X=np.random.randn(1,n_sim)*2+1
J=5000
f=[]
integrals=np.zeros(n_sim)
for i in range(n_sim):
    xi =np.random.uniform(0, 1, (J,))
    integrand = lambda x: np.mean(1/(xi)*((1-xi)<=x))
    integrals[i]=integrate.quad(integrand, 0, 1)[0]
    avars=[]
    for j in range(J):
        avars.append(avar(X[0],M,h,gamma,lbd,n_sim,1-xi[j]))
    f.append(np.mean(avars,axis=0)-k*max((integrals[i]-1),0))
evars=np.max(f,axis=0)